# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (SparkSession
 .builder
 .appName("Reto 1")
 .master("local")
 .getOrCreate())

### Carga de CSV

In [2]:
trade_details = (spark
                 .read
                 .format("csv")
                 .option("header", "true")
                 .option("delimiter", ";")
                 .load("./data/trade_details.csv")
                 .cache()
                )

trade_details_snapshot = (spark
                          .read
                          .format("csv")
                          .option("header", "true")
                          .option("delimiter", ";")
                          .load("./data/trade_details_snapshot.csv")
                          .cache()
                         )

In [6]:
trade_details_snapshot1 = (trade_details_snapshot
                           .join(trade_details,"origin_trade_number")
                           .select(trade_details_snapshot["origin_contract_number"],
                                   trade_details_snapshot["origin_trade_number"],
                                   trade_details_snapshot["maturity"]
                                  )     
                          )

trade_details_snapshot.unpersist()
trade_details_snapshot1.cache().show()

+----------------------+-------------------+----------+
|origin_contract_number|origin_trade_number|  maturity|
+----------------------+-------------------+----------+
|              18622136|           19665185|2020-04-29|
|              18622136|           19665186|2020-12-30|
|              18724280|           19772399|2020-11-05|
|              18724280|           19772400|2021-11-05|
|              19883451|           20980932|2020-02-02|
|              19883451|           20980933|2020-01-19|
|              21622649|           22798004|2021-05-04|
|              21622649|           22798005|2021-05-11|
|              19622128|           20665177|2020-04-29|
|              19622128|           20665178|2020-12-30|
+----------------------+-------------------+----------+



In [12]:
from pyspark.sql.window import Window

w = Window.partitionBy("origin_contract_number").orderBy("maturity")
diff = F.col("maturity").cast("long") - F.lag("maturity", 1).over(w).cast("long")

trade_details_snapshot2 = (trade_details_snapshot1
                           .select(   
                               F.col("origin_contract_number"),
                               F.col("origin_trade_number"),
                               F.to_timestamp(F.col("maturity")).alias("maturity"))
                           .withColumn("diff", diff)
                           .where(F.col("diff") > 0)
                           .select(F.col("origin_trade_number").alias("trade_number"))
                          )

trade_details_snapshot1.unpersist()
trade_details_snapshot2.cache().show()     

+------------+
|trade_number|
+------------+
|    20665178|
|    22798005|
|    20980932|
|    19665186|
|    19772400|
+------------+



### Resultado:

In [28]:
resultado = (trade_details
             .join(trade_details_snapshot2, 
                   F.col("origin_trade_number") == F.col("trade_number"),
                   "left"
                  )    
             .where(~( (F.col("mfamily") == "CURR") & 
                     (F.col("mgroup") == "FXD") & 
                     (F.col("mtype") == "SWLEG")
                    )
                    | (F.col("mtype") == "SWLEG") 
                     & (F.col("trade_number") > 0) 
                   )                    
             .drop(F.col("trade_number") )
            )
                        

trade_details.unpersist()
trade_details_snapshot2.cache().unpersist()



DataFrame[trade_number: string]

In [34]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()[24][4] == "564367838")
assert(resultado.orderBy("origin_contract_number").collect()[19][5] == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()[16][5] == "NULL")

In [35]:
resultado.show(33)

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|    IRD|    CF| null|           20513130|              19433281|2021-07-06|
|    IRD|    CF| null|           20533916|              19453781|2023-07-06|

In [36]:
resultado.unpersist()

DataFrame[mfamily: string, mgroup: string, mtype: string, origin_trade_number: string, origin_contract_number: string, maturity: string]